# Project Introduction

As employees for the hypothetical oil mining company OilyGiant, our task is to find the best place for a new oil well.

In order to choose the location, we must...
- Collect the oil well parameters in the selected region: oil quality and volume of reserves.
- Build a model for predicting the volume of reserves in the new wells.
- Pick the oil wells with the highest estimated values.
- Pick the region with the highest total profit for the selected oil wells.

We have data on oil samples from three regions. Parameters of each oil well in the region are already known. Build a model that will help to pick the region with the highest profit margin. Analyze potential profit and risks using the Bootstrapping technique.

## Data Description

Geological exploration data for the three regions are stored in files:
- geo_data_0.csv
- geo_data_1.csv
- geo_data_2.csv

Each dataset contains the following fields:
- id — unique oil well identifier
- f0, f1, f2 — three features of potential oil well points (their specific meaning is unimportant, but the features themselves are significant)
- product — volume of reserves in the oil well (thousands of barrels).

## Project Conditions
- Only linear regression is suitable for model training (other models are not sufficiently predictable).
- When exploring the region, a study of 500 points is carried out; the best 200 points are picked for the profit calculation.
- The budget for development of 200 oil wells is 100 USD million.
- One barrel of raw materials brings in $4.5 USD of revenue. The revenue from one unit of product is $4,500 dollars (volume of reserves is in thousands of barrels).
- After the risk evaluation, keep only the regions with the risk of losses lower than 2.5%. 
- From the ones that fit the criteria, the region with the highest average profit should be selected.

The data is synthetic: contract details and well characteristics are not disclosed.

## Project Plan

1. Import the necessary libraries for the project. 
2. Download and prepare the data.
3. Designate the features and target for the model.
4. Split the data into training and validation sets for each region.
5. Scale the features for each model.
5. Train the models and make predictions.
6. Evaluate the models' RSME scores & compare them to the RSME score of each geo's mean target value.
7. Prepare the data for the profit calculation.
9. Calculate The profit for the best 200 well locations in each geo and recommend the best region for development.
10. Calculate risks and profit for each region.

### Import the Necessary Libraries

In [19]:
# import the necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
import numpy as np
from scipy import stats as st


### Download and Prepare the Data

In [2]:
# load the datasets into dataframes
geo0 = pd.read_csv('geo_data_0.csv')
geo1 = pd.read_csv('geo_data_1.csv')
geo2 = pd.read_csv('geo_data_2.csv')

# get info about each dataframe
print("geo 0 info:\n")
geo0.info()

print("\ngeo 1 info:\n")
geo1.info()

print("\ngeo 2 info:\n")
geo2.info()

# preview each dataframe
print("\ngeo 0 preview:", geo0.head())
print("\ngeo 1 preview:", geo1.head())
print("\ngeo 2 preview:", geo2.head())

# check each dataframe for duplicate rows
print("\ngeo 0 number of duplicates:", geo0.duplicated().sum())
print("geo 1 number of duplicates:", geo1.duplicated().sum())
print("geo 2 number of duplicates:", geo2.duplicated().sum())

geo 0 info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

geo 1 info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

geo 2 info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (

Our data looks as expected with the correct data types. There are also no missing zeroes or duplicate rows.

### Designate the Features and Target for Each Model

In [3]:
# store the features in a separate dataframe for each region
features_geo0 = geo0.drop(['product', 'id'], axis=1)
features_geo1 = geo1.drop(['product', 'id'], axis=1)
features_geo2 = geo2.drop(['product', 'id'], axis=1)

# store the target in a separate series for each region
target_geo0 = geo0['product'] 
target_geo1 = geo1['product']
target_geo2 = geo2['product']


### Split the Data into Training and Validation Sets for Each Region

In [4]:
'''
Split the features and target into training and validation sets for each region.
''' 

# geo0
features_train_geo0, features_valid_geo0, target_train_geo0, target_valid_geo0 = train_test_split(
    features_geo0, target_geo0, test_size=0.25, random_state=12345)

# geo1
features_train_geo1, features_valid_geo1, target_train_geo1, target_valid_geo1 = train_test_split(
    features_geo1, target_geo1, test_size=0.25, random_state=12345)

# geo2
features_train_geo2, features_valid_geo2, target_train_geo2, target_valid_geo2 = train_test_split(
    features_geo2, target_geo2, test_size=0.25, random_state=12345)


For each region, we made the validation set 25% of the original dataset size.

### Scale the Features for Each Model

In [5]:
'''
geo0
'''

# initialize the StandardScaler
scaler_geo0 = StandardScaler()

# train the scaler and transform the training features for geo0
features_train_geo0 = scaler_geo0.fit_transform(features_train_geo0)
features_valid_geo0 = scaler_geo0.transform(features_valid_geo0)

'''
geo1
'''

# initialize the StandardScaler
scaler_geo1 = StandardScaler()

# train the scaler and transform the training features for geo1
features_train_geo1 = scaler_geo1.fit_transform(features_train_geo1)
features_valid_geo1 = scaler_geo1.transform(features_valid_geo1)

'''
geo2
'''

# initialize the StandardScaler
scaler_geo2 = StandardScaler()

# train the scaler and transform the training features for geo2
features_train_geo2 = scaler_geo2.fit_transform(features_train_geo2)
features_valid_geo2 = scaler_geo2.transform(features_valid_geo2)

### Train the Model and Make Predictions for Each Region

In [6]:
'''
Geo0
'''

# initialize the model constructor
model_geo0 = LinearRegression() 

# train the model on the training set
model_geo0.fit(features_train_geo0, target_train_geo0) 

# get model predictions on the validation set
predictions_valid_geo0 = model_geo0.predict(features_valid_geo0) 

'''
Geo1
'''

# initialize the model constructor
model_geo1 = LinearRegression() 

# train the model on the training set
model_geo1.fit(features_train_geo1, target_train_geo1) 

# get model predictions on the validation set
predictions_valid_geo1 = model_geo1.predict(features_valid_geo1) 

'''
Geo2
'''

# initialize the model constructor
model_geo2 = LinearRegression() 

# train the model on the training set
model_geo2.fit(features_train_geo2, target_train_geo2) 

# get model predictions on the validation set
predictions_valid_geo2 = model_geo2.predict(features_valid_geo2) 

In [7]:
''' 
For each region, reset the index of the validation set target values dataframe and create a series of the validation set predictions with the same index.
'''

# geo0
target_valid_geo0.reset_index(drop=True, inplace=True)
predictions_valid_geo0 = pd.Series(predictions_valid_geo0, index=target_valid_geo0.index)

# geo1
target_valid_geo1.reset_index(drop=True, inplace=True)
predictions_valid_geo1 = pd.Series(predictions_valid_geo1, index=target_valid_geo1.index)

# geo2
target_valid_geo2.reset_index(drop=True, inplace=True)
predictions_valid_geo2 = pd.Series(predictions_valid_geo2, index=target_valid_geo2.index)



### Evaluate the Model's RSME score & Compare it to the RSME Score of the Mean Target Value

In [18]:
# calculate the RMSE of the models on the validation set and print them
result_geo0 = mean_squared_error(target_valid_geo0, predictions_valid_geo0)**0.5
print("RMSE of the linear regression model on the validation set (geo0):", round(result_geo0,2))

result_geo1 = mean_squared_error(target_valid_geo1, predictions_valid_geo1)**0.5
print("RMSE of the linear regression model on the validation set (geo1):", round(result_geo1,2))

result_geo2 = mean_squared_error(target_valid_geo2, predictions_valid_geo2)**0.5
print("RMSE of the linear regression model on the validation set (geo2):", round(result_geo2,2))

# print the average value of the predictions for each region
print("\nAverage value of the predictions for geo0:", round(predictions_valid_geo0.mean(),2))
print("Average value of the predictions for geo1:", round(predictions_valid_geo1.mean(),2))
print("Average value of the predictions for geo2:", round(predictions_valid_geo2.mean(),2))

# print the RSME of comparing the mean of the target variable to the target variable for each region
print("\nRMSE of comparing the mean of the target variable to the target variable (geo0):",     round(root_mean_squared_error(target_geo0, pd.Series(target_geo0.mean(), index=target_geo0.index)), 2))
print("RMSE of comparing the mean of the target variable to the target variable (geo1):", round(root_mean_squared_error(target_geo1, pd.Series(target_geo1.mean(), index=target_geo1.index)),2))
print("RMSE of comparing the mean of the target variable to the target variable (geo2:", round(root_mean_squared_error(target_geo2, pd.Series(target_geo2.mean(), index=target_geo2.index)),2))


# print the max, min, and mean values of the target variable for each region
print("\nMax, min, and mean value of the target variable (geo0):", round(target_geo0.max(),2),",",target_geo0.min(),",",round(target_geo0.mean(),2))
print("Max, min, and mean value of the target variable (geo1):", round(target_geo1.max(),2),",", target_geo1.min(),",", round(target_geo1.mean(),2))
print("Max, min, and mean value of the target variable (geo2):", round(target_geo2.max(),2),",", target_geo2.min(),",", round(target_geo2.mean(),2))


RMSE of the linear regression model on the validation set (geo0): 37.58
RMSE of the linear regression model on the validation set (geo1): 0.89
RMSE of the linear regression model on the validation set (geo2): 40.03

Average value of the predictions for geo0: 92.59
Average value of the predictions for geo1: 68.73
Average value of the predictions for geo2: 94.97

RMSE of comparing the mean of the target variable to the target variable (geo0): 44.29
RMSE of comparing the mean of the target variable to the target variable (geo1): 45.94
RMSE of comparing the mean of the target variable to the target variable (geo2: 44.75

Max, min, and mean value of the target variable (geo0): 185.36 , 0.0 , 92.5
Max, min, and mean value of the target variable (geo1): 137.95 , 0.0 , 68.83
Max, min, and mean value of the target variable (geo2): 190.03 , 0.0 , 95.0


For geo0 and geo2, the RMSE value is around 20-21% of their respective targets' value range (min to max range). 

Geo1 has a very low RMSE score of about 0.89, which is great. 

We don't have a specific criteria about what our RSME score for our models must be, but the mean prediction value of each is very close to its respective mean (actual) target value, which is good.

As a sanity check, we can also confirm that each model's RMSE score performs better than chance i.e. using the same answer (each model's actual target mean value) for each model's target predictions.


### Prepare the Data for the Profit Calculation

In [9]:
# define the profit calculation inputs
total_budget = 100000000
number_of_wells = 200
budget_per_well = total_budget / number_of_wells
revenue_per_unit = 4500
break_even_units = budget_per_well / revenue_per_unit

print(f"Break-even point in product units per well: {break_even_units:.2f}")
print(f"Break-even point in required profit per well (USD): {budget_per_well:.2f}")

Break-even point in product units per well: 111.11
Break-even point in required profit per well (USD): 500000.00


For a region to break-even on the production of 200 new oil wells, an average well would have to produce 111.11 units of product (each unit represents a thousand barrels).

On average, regions 0, 1, and 2 produce 92.5, 68.8, and 95.0 units of product, respectively. Based on these averages alone, none of our regions would allow us to break even on our investment.

### Calculate The Profit for the Top 200 Wells in Each Region

In [10]:
# define a function to calculate the profit for the best 200 oil wells in geo0
def profit(target, predictions, count):
    probs_sorted = predictions.sort_values(ascending=False) # sort the values of our predictions in descending order
    selected = target[probs_sorted.index][:count] # select the target values with the top n predictions (n = count)
    return (revenue_per_unit * selected.sum()) - total_budget # multiple the revenue (USD) per unit of product by the units produced by the top n wells then subtract the total budget (USD)

# calculate the profit for the best 200 oil wells in geo0 and print the total profit and avg. profit per well
profit_geo0 = profit(target_valid_geo0, predictions_valid_geo0, number_of_wells)
print(f"Profit for the best 200 oil wells in geo0: {round(profit_geo0,2)}")
print(f"Average profit per well in geo0: {round((profit_geo0 / number_of_wells),2)}")

# calculate the profit for the best 200 oil wells in geo1 and print the total profit and avg. profit per well
profit_geo1 = profit(target_valid_geo1, predictions_valid_geo1, number_of_wells)
print(f"\nProfit for the best 200 oil wells in geo1: {round(profit_geo1,2)}")
print(f"Average profit per well in geo1: {round((profit_geo1 / number_of_wells),2)}")

# calculate the profit for the best 200 oil wells in geo2 and print the total profit and avg. profit per well
profit_geo2 = profit(target_valid_geo2, predictions_valid_geo2, number_of_wells)
print(f"\nProfit for the best 200 oil wells in geo2: {round(profit_geo2,2)}")
print(f"Average profit per well in geo2: {round((profit_geo2 / number_of_wells),2)}")


Profit for the best 200 oil wells in geo0: 33208260.43
Average profit per well in geo0: 166041.3

Profit for the best 200 oil wells in geo1: 24150866.97
Average profit per well in geo1: 120754.33

Profit for the best 200 oil wells in geo2: 27103499.64
Average profit per well in geo2: 135517.5


Geo0 is the most profitable overall with $33.2M in total profit and an average of $166.9K in profit per each of its top 200 wells (in terms of the wells that our model predicted would produce the most oil).

Geo0 is about $9M (+27%) more profitable than geo1 and about $6.1M (+18.4%) more profitable than geo2 overall. Without yet taking our risk tolerance into account, geo0 is the clear choice for where we'd recommend the company to develop more oil wells.

### Calculate Risks and Profit for Each Region

In [11]:
''' 
Use the bootstrapping technique to find the distribution of profit for each region.
'''

# set the RandomState value and store it in the variable "state"
state = np.random.RandomState(12345)

# set the number of bootstrap samples to 1000
bootstrap_samples = 1000


# def a function to generate bootstrap samples and calculate the samples' profit values, mean profit, 95% confidence interval for profit, and the probability of a loss
def bootstrap_profit_ci_risk(target, predictions, num_samples, samp_size, num_wells, profit_values, alpha):
    profit_values = [] # initiate an empty list to store the profit values
    for i in range(num_samples): # iterate over the number of samples
        target_subsample = target.sample(n=samp_size, replace=True, random_state=state) # generate a subsample of the target values
        pred_subsample = predictions[target_subsample.index] # generate a subsample of the predictions
    
        profit_values.append(profit(target_subsample, pred_subsample, num_wells)) # calculate the profit for the subsample and append it to the profit_values list

    profit_values = pd.Series(profit_values) # convert the profit_values list to a pandas series
    
    # designate the input values for the confidence interval calculation
    alpha = alpha
    df = len(profit_values) - 1
    loc = profit_values.mean()
    scale = profit_values.sem()

    # calculate the 95% confidence interval for the profit_values list and format the output
    confidence_interval = st.t.interval(alpha, df, loc, scale)
    confidence_interval = (float(confidence_interval[0]), float(confidence_interval[1]))
    
    # calculate the probability of a loss
    losses = profit_values[profit_values < 0]  # store profit values that are less than 0 in a separate series
    number_of_losses = len(losses) # calculate the number of losses
    total_samples = len(profit_values) # calculate the total number of samples
    probability_of_loss = number_of_losses / total_samples # calculate the probability of a loss

    # calculate the mean profit value
    mean_profit = round(float(profit_values.mean()),2) # calculate the mean profit

    # print the mean profit, 95% confidence interval for profit, and the probability of a loss
    return print('mean profit:', mean_profit), print('95% confidence interval for the region:', confidence_interval), print(f'Probability of a loss: {probability_of_loss:.1%}')


'''
Run the bootstrap_profit_ci_risk function for geo0.
'''

# initiate an empty list of values
values_geo0 = []

# run the bootstrap_profit function for geo0
print("Geo0:")
bootstrap_profit_ci_risk(target_valid_geo0, predictions_valid_geo0, bootstrap_samples, 500, 200, values_geo0, 0.95)

'''
Run the bootstrap_profit_ci_risk function for geo1.
'''

# initiate an empty list of values
values_geo1 = []

# run the bootstrap_profit function for geo1
print("\nGeo1:")
bootstrap_profit_ci_risk(target_valid_geo1, predictions_valid_geo1, bootstrap_samples, 500, 200, values_geo1, 0.95)

'''
Run the bootstrap_profit_ci_risk function for geo2.
'''

# initiate an empty list of values
values_geo2 = []

# run the bootstrap_profit function for geo2
print("\nGeo2:")
bootstrap_profit_ci_risk(target_valid_geo2, predictions_valid_geo2, bootstrap_samples, 500, 200, values_geo2, 0.95)

print("\n")


Geo0:
mean profit: 4259385.27
95% confidence interval for the region: (4087322.0706869857, 4431448.467524861)
Probability of a loss: 6.0%

Geo1:
mean profit: 5182594.94
95% confidence interval for the region: (5052498.815766218, 5312691.058180281)
Probability of a loss: 0.3%

Geo2:
mean profit: 4201940.05
95% confidence interval for the region: (4025287.0365036144, 4378593.070377386)
Probability of a loss: 6.2%




Geo1 is where we should develop new oil wells. It has the highest mean profit of $5.2M, the highest range of profit values for its 95% confidence interval range, and it's also the only region with less than a 2.5% probability of realizing a loss, which is our maximum tolerance for risk.

To explore a region, the oil company studies 500 potential oil well locations and pick the best 200 to develop. The bootstrapping technique allowed us to simulate these studies by taking 1,000 samples of 500 potential oil sites, selecting the best 200 sites out of the 500 potential sites for each sample and, finally, calculating the profit each sample would generate.

From there, we were able to calculate the average profitability of each region based on its 1,000 sample results, which gives us an average value that would be very close to the true population mean. We also were able to calculate the 95% confidence interval for the distribution of profit values that our 1,000 bootstrap samples gave us; this gives us confidence that, for geo1 as an example, we'd have a 95% probability of generating a range of profit from $5.05M to $5.31 in geo 1 no matter which 500 sites we decided to study and develop the top 200 sites within.

# Project Conclusion

If we relied on the linear regression models for each region alone, geo0 would have been the best region to develop new oil wells in with a total predicted profit of $33.2M in profit. 

However, the bootstraping technique allowed us to replicate the same conditions that the company employs when exporing a region i.e. developing the best 200 sites out of a given 500 potential sites that they study. Additionally, the bootstapping technique allowed us to make a recommendation that satisfied our maximum risk tolerance of 2.5% for realizing a loss. 

Ultimately, after applying the bootstrapping technique, geo1 was the only region that met our risk tolerance criteria. Fortunately, it also returned the highest average profit value of about $5.2M USD. With a 95% probability, we can feel confident that developing 200 new oil wells in geo1 will bring in a range of $5.05M to $5.31M USD and that there is only about a 0.3% probability of realizing a loss from the initiative. Notably, before we applied the bootstrapping technique, the geo1 region's linear regression model had the lowest RMSE value, which seems to align with its lowest risk of realizing a loss due to the lower variability of each site's production volume from the region's production volume mean in both actuality and in terms of our model's predicted production levels.